In [ ]:
import pandas as pd
import pyarrow as pa

In [ ]:
train = pd.read_csv('train_events.csv')
train

In [ ]:
train['region'] = train['region'].replace(r"[’'`]", '', regex=True)

In [ ]:
video_info = pd.read_csv('video_info_v3.csv')
targets = pd.read_csv("train_targets.csv")
features = pd.read_csv("features.csv")

In [ ]:
targets = targets.drop(['age'], axis=1)

In [ ]:
targets['sex'] = targets['sex'].map({"male": 1, "female": 0})
targets

In [ ]:
train = train.merge(video_info[['rutube_video_id', 'category', 'duration', 'author_id', 'label']], how="inner",
                    on='rutube_video_id')
train

In [ ]:
len(video_info['author_id'].unique())

In [ ]:
train = train.merge(features, how='left', on='author_id')
train

In [ ]:
train['ua_os'] = train['ua_os'].fillna("Undefined")
train

In [ ]:
sx = train['sex_mode'].value_counts().idxmax()
mn = train['median_age'].mean()
train['sex_mode'] = train['sex_mode'].fillna(sx)
train['median_age'] = train['median_age'].fillna(mn)
train

In [ ]:
train['sex_mode'] = train['sex_mode'].map({"male": 1, "female": 0})
train['median_age'] = train['median_age'].astype(int)
train

In [ ]:
from datetime import timedelta

train['event_timestamp'] = pd.to_datetime(train['event_timestamp'])
train['event_timestamp'] = train['event_timestamp'] - timedelta(hours=3) # 6-12 утро 13-18 день 19-0 вечер 1-5 ночь
train

In [ ]:
time_zones = {
    'Tatarstan Republic': 3,
    'Bashkortostan Republic': 5,
    'Novosibirsk Oblast': 7,
    'Moscow': 3,
    'Moscow Oblast': 3,
    'Omsk Oblast': 7,
    'Chuvashia': 3,
    'Krasnoyarsk Krai': 7,
    'Chelyabinsk': 5,
    'Kamchatka': 11,
    'Nizhny Novgorod Oblast': 3,
    'Krasnodar Krai': 3,
    'Volgograd Oblast': 3,
    'Kaliningrad Oblast': 2,
    'Kuzbass': 7,
    'Stavropol Kray': 3,
    'Samara Oblast': 4,
    'Amur Oblast': 9,
    'Sverdlovsk Oblast': 5,
    'St.-Petersburg': 3,
    'Yamalo-Nenets': 5,
    'Orenburg Oblast': 5,
    'Khanty-Mansia': 5,
    'Kaluga Oblast': 3,
    'Tomsk Oblast': 7,
    'Novgorod Oblast': 3,
    'Arkhangelskaya': 3,
    'North Ossetia–Alania': 3,
    'Kursk Oblast': 3,
    "Leningradskaya Oblast'": 3,
    'Krasnoyarskiy': 7,
    'Ivanovo Oblast': 3,
    'Altay Kray': 7,
    'Kurgan Oblast': 5,
    'Kostroma Oblast': 3,
    'Bryansk Oblast': 3,
    'Dagestan': 3,
    'Lipetsk Oblast': 3,
    'Vladimir Oblast': 3,
    'Kirov Oblast': 3,
    'Khabarovsk': 9,
    'Tambov Oblast': 3,
    'Chukotka': 12,
    'Voronezh Oblast': 3,
    'Sverdlovsk': 5,
    'Tula Oblast': 3,
    'Krasnodarskiy': 3,
    'Irkutsk Oblast': 8,
    'Saratov Oblast': 4,
    'Khakasiya Republic': 7,
    'Penza': 3,
    'Perm Krai': 5,
    'Oryol oblast': 3,
    'Vladimir': 3,
    'Smolensk Oblast': 3,
    'Penza Oblast': 3,
    'Mordoviya Republic': 3,
    'Tyumen’ Oblast': 5,
    'Sakha': 9,
    'Primorye': 10,
    'Zabaykalskiy (Transbaikal) Kray': 9,
    'Vologda Oblast': 3,
    'Yaroslavl Oblast': 3,
    'Crimea': 3,
    'Rostov': 3,
    'Ryazan Oblast': 3,
    'Perm': 5,
    'Chechnya': 3,
    'Udmurtiya Republic': 3,
    'Tver Oblast': 3,
    'Buryatiya Republic': 8,
    'Belgorod Oblast': 3,
    'Kaluga': 3,
    'Astrakhan Oblast': 3,
    'Karelia': 3,
    'Murmansk': 3,
    'Adygeya Republic': 3,
    'Kemerovo Oblast': 7,
    'Mariy-El Republic': 3,
    'Kursk': 3,
    'Saratovskaya Oblast': 4,
    'Sakhalin Oblast': 10,
    'Ivanovo': 3,
    'Tyumen Oblast': 5,
    'Stavropol’ Kray': 3,
    'Voronezj': 3,
    'Karachayevo-Cherkesiya Republic': 3,
    'Kabardino-Balkariya Republic': 3,
    'Ulyanovsk': 4,
    'North Ossetia': 3,
    'Komi': 3,
    'Smolensk': 3,
    'Tver’ Oblast': 3,
    'Sebastopol City': 3,
    'Pskov Oblast': 3,
    'Tula': 3,
    'Orel Oblast': 3,
    'Jaroslavl': 3,
    'Tambov': 3,
    'Kalmykiya Republic': 3,
    'Primorskiy (Maritime) Kray': 10,
    'Altai': 7,
    'Magadan Oblast': 10,
    'Vologda': 3,
    'Tyva Republic': 7,
    'Nenets': 3,
    'Smolenskaya Oblast': 3,
    'Jewish Autonomous Oblast': 10,
    'Astrakhan': 3,
    'Ingushetiya Republic': 3,
    'Kirov': 3,
    'Transbaikal Territory': 9,
    'Omsk': 7,
    'Kaliningrad': 2,
    'Stavropol Krai': 3,
    'Arkhangelsk Oblast': 3,
    'Leningradskaya Oblast': 3
}

In [ ]:
train['timezone_offset'] = train['region'].map(time_zones)
train

In [ ]:
train['event_timestamp_timezone'] = train['event_timestamp'] + pd.to_timedelta(train['timezone_offset'], unit='h')
train

In [ ]:
train['morning'] = 0
train['day'] = 0
train['evening'] = 0
train['night'] = 0

In [ ]:
train.loc[train['event_timestamp_timezone'].dt.hour.between(6, 12, inclusive='left'), 'morning'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(13, 18, inclusive='left'), 'day'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(19, 23, inclusive='left'), 'evening'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(0, 5, inclusive='both'), 'night'] = 1
train

In [ ]:
category = video_info['category'].unique()
cat_dict = {c: c_id for c_id, c in enumerate(category)}

In [ ]:
train['category'] = train['category'].map(cat_dict)
train

In [ ]:
devices = train['ua_device_type'].unique()
device_dict = {dev: dev_i for dev_i, dev in enumerate(devices)}
train['ua_device_type'] = train['ua_device_type'].map(device_dict)
train

In [ ]:
region_unique = train['region'].unique()
region_dict = {reg: reg_i for reg_i, reg in enumerate(region_unique)}
region_dict

In [ ]:
train['region'] = train['region'].map(region_dict)
train

In [ ]:
train.loc[train['viewer_uid']==10067243, 'region'][0]

In [ ]:
category_cols = [f"category_{i}" for i in range(len(category))]
label_cols = [f"label_{i}" for i in range(100)]
device_cols = [f"device_{i}" for i in range(len(train['ua_device_type'].unique()))]

In [ ]:
device_cols

In [ ]:
train['percent'] = train['total_watchtime'] / train['duration']
train

# Targets

In [ ]:
usr_emb = targets.copy()
usr_emb

In [ ]:
cat_df = pd.DataFrame(columns=['viewer_uid', *category_cols])
cat_df

In [ ]:
label_df = pd.DataFrame(columns=['viewer_uid', *label_cols])
label_df

In [ ]:
device_df = pd.DataFrame(columns=['viewer_uid', *device_cols])
device_df

In [ ]:
label_counts = train.pivot_table(index='viewer_uid', columns='label', aggfunc='size', fill_value=0)

for la in range(100):
    if la not in label_counts.columns:
        label_counts[la] = 0

label_counts = label_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(100)))

label_counts.columns = ['viewer_uid'] + label_cols

label_df = label_counts
label_df

In [ ]:
category_counts = train.pivot_table(index='viewer_uid', columns='category', aggfunc='size', fill_value=0)

for category in range(40):
    if category not in category_counts.columns:
        category_counts[category] = 0

category_counts = category_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(40)))

category_counts.columns = ['viewer_uid'] + category_cols

cat_df = category_counts
cat_df

In [ ]:
device_counts = train.pivot_table(index='viewer_uid', columns='ua_device_type', aggfunc='size', fill_value=0)

for dev in range(3):
    if dev not in device_counts.columns:
        device_counts[dev] = 0

device_counts = device_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(3)))

device_counts.columns = ['viewer_uid'] + device_cols

device_df = device_counts
device_df

In [ ]:
usr_emb

In [ ]:
t = pa.Table.from_pandas(train)

In [ ]:
t.to_pandas()

In [ ]:
sex_mean = t.select(['viewer_uid', 'sex_mode']).group_by(
    ['viewer_uid']).aggregate([('sex_mode', "mean")])
sex_mean = sex_mean.to_pandas()
sex_mean

In [ ]:
time_sum = t.select(['viewer_uid', 'morning', 'day','evening', 'night']).group_by(['viewer_uid']).aggregate([('morning', "sum"), ('day', "sum"), ('evening', "sum"), ('night', "sum")])
time_sum = time_sum.to_pandas()
time_sum

In [ ]:
median = train.groupby('viewer_uid')['median_age'].agg(list)

In [ ]:
median_df = pd.DataFrame({'viewer_uid': median.index, 'median_age': median.values})

In [ ]:
from statistics import median

median_df['median_age'] = median_df['median_age'].apply(lambda x: median(x))
median_df['median_age'] = median_df['median_age'].astype(int)
median_df

In [ ]:
region_map = train.drop_duplicates('viewer_uid').set_index('viewer_uid')['region'].to_dict()
usr_emb['region'] = usr_emb['viewer_uid'].map(region_map)
usr_emb

In [ ]:
usr_emb = usr_emb.merge(cat_df, how='inner', on='viewer_uid')

In [ ]:
usr_emb = usr_emb.merge(median_df, how='inner', on='viewer_uid')

In [ ]:
usr_emb = usr_emb.merge(sex_mean, how='inner', on='viewer_uid')

In [ ]:
usr_emb = usr_emb.merge(label_df, how='inner', on='viewer_uid')

In [ ]:
usr_emb = usr_emb.merge(device_df, how='inner', on='viewer_uid')

In [ ]:
usr_emb = usr_emb.merge(time_sum, how='inner', on='viewer_uid')

In [ ]:
usr_emb

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(usr_emb.drop(['sex', 'age_class', 'viewer_uid'], axis=1),
                                                    usr_emb['sex'], test_size=0.33, random_state=42)


In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=1500, eval_metric='Accuracy', learning_rate=0.1, max_depth=7,
                           early_stopping_rounds=250, task_type="GPU", leaf_estimation_backtracking='AnyImprovement')

In [ ]:
model.fit(x_train, y_train, eval_set=(x_test, y_test), cat_features=['region'], use_best_model=True)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

acc = accuracy_score(y_test, y_pred)
acc

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(usr_emb.drop(['sex', 'age_class', 'viewer_uid'], axis=1),
                                                    usr_emb['age_class'], test_size=0.3, random_state=42)

In [ ]:
model = CatBoostClassifier(iterations=1500, eval_metric='Accuracy',loss_function='MultiClass', learning_rate=0.1, max_depth=6,
                           early_stopping_rounds=250, task_type="GPU", leaf_estimation_backtracking='AnyImprovement')

In [ ]:
model.fit(x_train, y_train, eval_set=(x_test, y_test),cat_features=['region'], use_best_model=True)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
f = f1_score(y_test, y_pred, average='weighted')

In [ ]:
f

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
import seaborn as sb

sb.pairplot(usr_emb)